### Fake News Classifier Using Bidirectional LSTM

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values
df=df.dropna()

In [5]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
# Get the Dependent features
y=df['label']

In [7]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [8]:
X.shape

(18285, 4)

In [9]:
y.shape

(18285,)

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'2.6.0'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\venky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[897, 4830, 2124, 1233, 3938, 1338, 4850, 3635, 2795, 4773],
 [4161, 2083, 301, 342, 124, 1959, 4195],
 [3043, 1650, 4034, 1310],
 [1772, 3725, 2503, 2175, 1692, 922],
 [182, 124, 3113, 11, 4749, 728, 124, 2152, 4278, 3844],
 [221,
  1876,
  2149,
  2776,
  4101,
  3466,
  1033,
  2160,
  906,
  169,
  3319,
  1893,
  3755,
  1629,
  4195],
 [3785, 3811, 658, 1933, 3506, 4241, 3661, 2199, 4961, 3145, 318],
 [3884, 366, 3537, 2946, 863, 4389, 3466, 2909, 4961, 3145, 318],
 [1481, 1160, 4246, 4306, 3896, 2304, 3621, 3004, 3466, 1914],
 [3727, 70, 758, 4172, 1022, 3809, 2176, 1740],
 [2798, 4042, 2807, 2209, 916, 2996, 168, 4529, 3668, 2614, 2064],
 [2175, 2617, 3938, 2304, 3466, 863],
 [2392, 27, 1545, 1747, 2893, 958, 187, 2908, 948],
 [1907, 2425, 4911, 2172, 4405, 491, 1559, 4961, 3145, 318],
 [1548, 3065, 2124, 3235, 4204, 4961, 3145, 318],
 [4568, 3092, 929, 4614, 1178, 4194, 3636, 1643, 252, 114],
 [3752, 1687, 2083],
 [4902, 3308, 513, 3547, 3466, 3667, 276, 4195],
 [1955, 4, 301

### Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3635 2795 4773]
 [   0    0    0 ...  124 1959 4195]
 [   0    0    0 ... 1650 4034 1310]
 ...
 [   0    0    0 ... 4961 3145  318]
 [   0    0    0 ... 1075 4401  904]
 [   0    0    0 ... 4797 4073  280]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  897,
       4830, 2124, 1233, 3938, 1338, 4850, 3635, 2795, 4773])

In [24]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs),y.shape

(18285, (18285,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [30]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 28ms/step - loss: 0.3061 - accuracy: 0.8527 - val_loss: 0.1913 - val_accuracy: 0.9188
Epoch 2/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1329 - accuracy: 0.9496 - val_loss: 0.2141 - val_accuracy: 0.9228
Epoch 3/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0923 - accuracy: 0.9679 - val_loss: 0.2732 - val_accuracy: 0.9158
Epoch 4/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0695 - accuracy: 0.9766 - val_loss: 0.2663 - val_accuracy: 0.9157
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0516 - accuracy: 0.9839 - val_loss: 0.3183 - val_accuracy: 0.9142
Epoch 6/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0361 - accuracy: 0.9890 - val_loss: 0.3891 - val_accuracy: 0.9049
Epoch 7/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0285 - accuracy: 0.9917 - val_loss: 0.3913 - val_accuracy: 0.9097

### Performance Metrics and Accuracy

In [31]:
y_pred1 = ((model1.predict(X_test)>0.5).astype("int32"))

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test, y_pred1)

array([[3114,  305],
       [ 246, 2370]], dtype=int64)

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

NameError: name 'y_pred' is not defined

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))